In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from src.robot_reboot.dataset_factory import RobotRebootDataSetFactory

In [2]:
cnn = tf.keras.models.load_model('model/model_0')
cnn.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 31, 31, 9)]  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 31, 31, 64)   9280        input[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 15, 15, 64)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 15, 15, 64)   65600       max_pooling2d_6[0][0]            
____________________________________________________________________________________________

In [3]:
ds_factory = RobotRebootDataSetFactory(31, cnn, max_depth=20, seed=26, playouts=50)

In [4]:
rows, cols, layers = 31, 31, 9
robots = int((layers - 1)/2)
p_columns = [f'p_{i}' for i in range(robots * 4)]
maze_columns = [f'm_{i}' for i in range(rows * cols *layers)]
columns = ['v'] + p_columns + maze_columns

In [5]:
def save(v, p, s, i):
    p_and_maze = np.concatenate((p, s.reshape(1, rows * cols * layers)), axis=None)
    data = np.concatenate((np.array([v]), p_and_maze), axis=None).reshape(1, 8666)
    df = pd.DataFrame(data = data, columns = columns)
    print(f'Saving data/ds_{i}.csv')
    df.to_csv(f'data/ds_{i}.csv')

In [6]:
def create(ds_factory, max_movements, i):
    print(f'Creating sample: {i}')
    v, p, s = ds_factory.create(locate_robot_close_goal=True, max_movements=max_movements)
    save(v, p, s, i)

In [ ]:
from threading import Thread
from time import sleep

np.random.seed(26)
threads = list()
for i in range(0, 2):
    if i% 20 ==0 and i!=0:
        print(f'Created {i}/600')
        [t.join() for t in threads]
        threads = list()
    max_movements = np.random.randint(1, 6)
    thread = Thread(target = create, args=(ds_factory, max_movements, i))
    thread.start()
    threads.append(thread)
[t.join() for t in threads]

Creating 0-th sample
Creating 1-th sample
Saving data/ds_1.csv


In [ ]:
np.random.randint(1, 6)

In [ ]:
df = pd.read_csv('data/df_0')

In [ ]:
df.head

In [ ]:
df[p_columns]